# C964 Predictive Model Training & Evaluation

## Setup

In [112]:
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import loguniform
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import (
    HistGradientBoostingClassifier,
    HistGradientBoostingRegressor,
    RandomForestClassifier,
    RandomForestRegressor,
)
from sklearn.impute import SimpleImputer
from sklearn.linear_model import (
    LinearRegression,
    LogisticRegression,
    PoissonRegressor,
    RANSACRegressor,
)
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    classification_report,
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    median_absolute_error,
    r2_score,
)
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.multioutput import (
    ClassifierChain,
    MultiOutputClassifier,
    MultiOutputRegressor,
)
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    StandardScaler,
)
from sklearn.svm import SVC, SVR
from sklego.meta import ZeroInflatedRegressor

%matplotlib inline

sns.set_style("darkgrid")
sns.set_palette(sns.color_palette("mako"))
warnings.simplefilter(action="ignore", category=FutureWarning)

In [113]:
# Features file manually encodes the variable type for each selected column for use in training
meta = pd.read_csv("./data/features.csv")
meta

,variable,kind
0,HDD65,numerical
1,CDD65,numerical
2,TYPEHUQ,categorical
3,CELLAR,boolean
4,BASEFIN,boolean
...,...,...
89,EVCHRGHOME,boolean
90,NHSLDMEM,numerical
91,SQFTEST,numerical
92,DBT1,numerical


In [114]:
# Categorize variable by type
numerical_vars = meta.query("`kind` == 'numerical'")["variable"].tolist()
boolean_vars = meta.query("`kind` == 'boolean'")["variable"].tolist()
ordinal_vars = meta.query("`kind` == 'ordinal'")["variable"].tolist()
categorical_vars = meta.query("`kind` == 'categorical'")["variable"].tolist()

# Define target and feature variables
target_vars = ["BTUEL", "BTUNG", "BTULP", "BTUFO", "BTUWD"]
feature_vars = [col for col in list(meta["variable"]) if col not in target_vars]

# Assign features by type
numerical_features = [col for col in numerical_vars if col in feature_vars]
boolean_features = [col for col in boolean_vars if col in feature_vars]
ordinal_features = [col for col in ordinal_vars if col in feature_vars]
categorical_features = [col for col in categorical_vars if col in feature_vars]

# Display counts of features by type
print("numerical_features", len(numerical_features))
print("boolean_features", len(boolean_features))
print("ordinal_features", len(ordinal_features))
print("categorical_features", len(categorical_features))
print("targets", len(target_vars))

numerical_features 36
boolean_features 22
ordinal_features 6
categorical_features 30
targets 5


In [115]:
# Define Pandas datatypes for variables
dtype = {}
for col in categorical_vars + ordinal_vars:
    dtype[col] = "category"
for col in boolean_vars:
    dtype[col] = "boolean"
for col in numerical_vars:
    dtype[col] = "float64"

# Import survey response dataset
data = pd.read_csv(
    "./data/recs2020_public_v5.csv",
    usecols=(feature_vars + target_vars),
    dtype=dtype,
    na_values=["-2"],
)

# Create imputed classification targets
data["USENG"] = data["BTUNG"] > 0
data["USELP"] = data["BTULP"] > 0
data["USEFO"] = data["BTUFO"] > 0
data["USEWD"] = data["BTUWD"] > 0

target_vars.extend(["USENG", "USELP", "USEFO", "USEWD"])

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data[feature_vars], data[target_vars], test_size=0.2, random_state=42
)
X_train

,HDD65,CDD65,TYPEHUQ,CELLAR,BASEFIN,ATTIC,ATTICFIN,STORIES,SIZEOFGARAGE,YEARMADERANGE,...,H2OAPT,WHEATSIZ,FUELH2O,MORETHAN1H2O,FUELH2O2,EVCHRGHOME,NHSLDMEM,SQFTEST,DBT1,DBT99
18412,2858.0,2034.0,5,<NA>,<NA>,<NA>,<NA>,NaN,NaN,2,...,True,1,5,False,NaN,<NA>,2.0,700.0,96.8,21.7
4909,4228.0,1539.0,2,False,<NA>,False,<NA>,1,NaN,4,...,<NA>,1,2,False,NaN,<NA>,3.0,1080.0,93.4,13.9
10031,3184.0,1594.0,3,False,<NA>,True,True,1,2,7,...,<NA>,2,1,False,NaN,<NA>,2.0,1710.0,90.4,20.9
9859,3061.0,1892.0,2,False,<NA>,True,False,2,NaN,4,...,<NA>,3,5,False,NaN,<NA>,3.0,2400.0,94.7,17.4
10359,3890.0,1647.0,2,False,<NA>,False,<NA>,2,2,5,...,<NA>,2,5,False,NaN,<NA>,1.0,1750.0,92.9,21.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,3179.0,1481.0,3,False,<NA>,True,False,1,2,7,...,<NA>,2,5,False,NaN,<NA>,2.0,1460.0,90.4,22.2
11964,9083.0,299.0,2,True,True,False,<NA>,1,NaN,6,...,<NA>,3,5,False,NaN,<NA>,4.0,2920.0,81.3,-12.5
5390,2549.0,1867.0,2,False,<NA>,True,False,2,2,6,...,<NA>,2,5,False,NaN,<NA>,2.0,2990.0,91.4,23.3
860,5776.0,648.0,2,True,False,True,True,1,NaN,1,...,<NA>,2,1,False,NaN,<NA>,2.0,1000.0,85.8,7.9


In [149]:
def create_constant_imputer(fill_value):
    return SimpleImputer(
        strategy="constant", fill_value=fill_value, add_indicator=False
    )


def create_ordinal_transformer(*, fill_value="0", categories="auto"):
    return make_pipeline(
        create_constant_imputer(fill_value), OrdinalEncoder(categories=categories)
    )


default_ordinal_transformer = create_ordinal_transformer()

preprocess_ordinal = ColumnTransformer(
    [
        (
            "SIZEOFGARAGE",
            create_ordinal_transformer(categories=[["0", "1", "2", "3"]]),
            ["SIZEOFGARAGE"],
        ),
        (
            "YEARMADERANGE",
            create_ordinal_transformer(
                categories=[["1", "2", "3", "4", "5", "6", "7", "8", "9"]]
            ),
            ["YEARMADERANGE"],
        ),
        (
            "TYPEGLASS",
            create_ordinal_transformer(categories=[["1", "2", "3"]]),
            ["TYPEGLASS"],
        ),
        (
            "ADQINSUL",
            create_ordinal_transformer(categories=[["1", "2", "3", "4"]]),
            ["ADQINSUL"],
        ),
        (
            "WASHTEMP",
            create_ordinal_transformer(categories=[["0", "1", "2", "3", "0"]]),
            ["WASHTEMP"],
        ),
        (
            "WHEATSIZ",
            create_ordinal_transformer(categories=[["1", "2", "3", "4"]]),
            ["WHEATSIZ"],
        ),
    ],
    remainder=default_ordinal_transformer,
)


preprocess_categorical = make_pipeline(
    OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False)
)


preprocess_boolean = make_pipeline(
    create_constant_imputer(fill_value=False),
    OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False),
)


normalized_transformer = make_pipeline(
    PowerTransformer(method="yeo-johnson", standardize=True)
)

standardized_transformer = make_pipeline(
    create_constant_imputer(fill_value=0), StandardScaler()
)

preprocess_numerical = ColumnTransformer(
    [("normalize", normalized_transformer, ["HDD65", "CDD65", "SQFTEST"])],
    remainder=standardized_transformer,
)

preprocessing = ColumnTransformer(
    [
        ("numerical", preprocess_numerical, numerical_features),
        ("boolean", preprocess_boolean, boolean_features),
        ("ordinal", preprocess_ordinal, ordinal_features),
        ("categorical", preprocess_categorical, categorical_features),
    ]
)
preprocessing

ColumnTransformer(transformers=[('numerical',
                                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                              SimpleImputer(fill_value=0,
                                                                                            strategy='constant')),
                                                                             ('standardscaler',
                                                                              StandardScaler())]),
                                                   transformers=[('normalize',
                                                                  Pipeline(steps=[('powertransformer',
                                                                                   PowerTransformer())]),
                                                                  ['HDD65',
                                                                   'CDD65',
                                                                   'SQFTEST'])]),
                                 ['HDD65', 'CDD65', 'BEDROOMS', 'NCOMBATH',
                                  'N...
                                 ['TYPEHUQ', 'STORIES', 'WALLTYPE', 'ROOFTYPE',
                                  'FUELPOOL', 'FUELTUB', 'TYPERFR1', 'TYPERFR2',
                                  'LOCRFRI2', 'UPRTFRZR', 'RANGEFUEL',
                                  'COOKTOPFUEL', 'OVENFUEL', 'OUTGRILLFUEL',
                                  'DWCYCLE', 'TOPFRONT', 'DRYRFUEL', 'EQUIPM',
                                  'FUELHEAT', 'EQUIPAUXTYPE', 'FUELAUX',
                                  'HUMIDTYPE', 'ACEQUIPM_PUB',
                                  'ACEQUIPAUXTYPE_PUB', 'DEHUMTYPE',
                                  'TYPETHERM', 'HEATCNTL', 'COOLCNTL',
                                  'FUELH2O', 'FUELH2O2'])])

In [187]:
# classifier = ClassifierChain(HistGradientBoostingClassifier(random_state=42, max_iter=10_000, early_stopping=True, warm_start=True), random_state=42)
# classifier_model = make_pipeline(preprocessing, classifier, memory='tmp/cache')
# classifier_model.fit(X_train, y_train[['USENG', 'USELP', 'USEFO', 'USEWD']])

# y_pred = classifier_model.predict(X_test)

# print(classification_report(
#     y_test[['USENG', 'USELP', 'USEFO', 'USEWD']],
#     y_pred,
#     target_names=['NG > 0', 'LP > 0', 'FO > 0', 'WD > 0'],
#     zero_division=np.nan,
#     digits=3,
# ))

Pipeline(memory='tmp/cache',
         steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                               SimpleImputer(fill_value=0,
                                                                                                             strategy='constant')),
                                                                                              ('standardscaler',
                                                                                               StandardScaler())]),
                                                                    transformers=[('normalize',
                                                                                   Pipeline(steps=[('powertransformer',
                                                                                                    PowerTransformer())]),
                                                                                   ['HDD65',
                                                                                    'CDD65'...
                                                   'DWCYCLE', 'TOPFRONT',
                                                   'DRYRFUEL', 'EQUIPM',
                                                   'FUELHEAT', 'EQUIPAUXTYPE',
                                                   'FUELAUX', 'HUMIDTYPE',
                                                   'ACEQUIPM_PUB',
                                                   'ACEQUIPAUXTYPE_PUB',
                                                   'DEHUMTYPE', 'TYPETHERM',
                                                   'HEATCNTL', 'COOLCNTL',
                                                   'FUELH2O', 'FUELH2O2'])])),
                ('classifierchain',
                 ClassifierChain(base_estimator=HistGradientBoostingClassifier(early_stopping=True,
                                                                               max_iter=10000,
                                                                               random_state=42,
                                                                               warm_start=True),
                                 random_state=42))])

In [195]:
class loguniform_int:
    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        return self._distribution.rvs(*args, **kwargs).astype(int)


features = data[feature_vars]

target_var = "BTUEL"
target = data[target_var]

model = make_pipeline(
    preprocessing,
    HistGradientBoostingRegressor(max_iter=10000, early_stopping=True, random_state=42),
)

param_dist = {
    "histgradientboostingregressor__l2_regularization": loguniform(1e-6, 1e3),
    "histgradientboostingregressor__learning_rate": loguniform(0.001, 1),
    "histgradientboostingregressor__max_leaf_nodes": loguniform_int(2, 256),
    "histgradientboostingregressor__min_samples_leaf": loguniform_int(1, 100),
    "histgradientboostingregressor__max_bins": loguniform_int(2, 255),
}

search = RandomizedSearchCV(
    model, param_dist, n_iter=20, n_jobs=2, verbose=1, random_state=42
)
cv = KFold(n_splits=5, shuffle=True, random_state=42)
results = cross_validate(
    search, features, target, cv=cv, n_jobs=2, return_estimator=True
)

cv_results = pd.DataFrame(cv_results)
cv_test_scores = cv_results["test_score"]
print(
    "Generalization score with hyperparameters tuning:\n"
    f"{cv_test_scores.mean():.3f} ± {cv_test_scores.std():.3f}"
)

/home/jon/dev/c964-model/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [27] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/jon/dev/c964-model/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [27] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Generalization score with hyperparameters tuning:
0.594 ± 0.018
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
